In [3]:
%Import the data
%file name is a string variable, includes extension

fname = 'Data1.xlsx';
Data = xlsread(fname);

In [4]:
%Find the correlations between columns to determine in this
%toy dataset which column.
%From excel worksheet will be he Y variable and which the 
%X variables.
%Output reads like the following:
%          col 1  col 2  col 3
%   col 1| 1.0   0.1328  0.4144
%   col 2|0.1328   1.0   0.0494
%   col 3|0.4144 0.0494    1.0

rho = corr(Data)

rho =

    1.0000    0.1328    0.4144
    0.1328    1.0000    0.0494
    0.4144    0.0494    1.0000


In [5]:
%Generate X and Y variables

%Data in the form of Data(:,1), where ':' is the place 
%holder for rows for Data(row,column).
%Data is an NxM dataset with N rows (our observations),
%and M columns.

%Number of data points:
%size function: size(Vector or Matrix, axis)
%axis=1 --> rows, axis=2 -->column
N=size(Data,1);

%Y variable: take the first column, M=1, and all rows
Y=Data(:,1);

%Create the X matrix

%First: 
%We need to create the vector of constants. 
%Use 'ones()' function: ones(size of first dim, size of 
%second dim, size of thid dim,..).
%For a 2-dimensional matrix, first argument is the row 
%dimension, second argument is the column dimension 
%(if it were a 3D matrix, third dimension is the 'height').
%Here we create an Nx1 vector

Constant = ones(N,1);

%Second:
%We can now concatinate: Our constant vector, and the two 
%other columns from the dataset.
%Data(:,2:end) --> grab all rows (:), and starting from 
%column 2 grab everything until the end.
%This creates an NxK matrix

X = [Constant Data(:,2:end)];

%Third: 
%For future, let's grab the number of regressors (call 
%it K:

K = size(X,2);

## The Linear Model We Will Estimate:

Here the model we will estimate is the following, abstracting from whatever interpretation we want to give to each regressor $x_{k}$.

The basic model is:

\begin{equation}
y_{i} = \beta_{0} + \beta_{1}x_{1,i} + \beta_{2}x_{2,i} + \varepsilon_{i}
\end{equation}

or in matrix algebra (when we have the N systems of equations):

\begin{equation}
\underbrace{Y}_{N\times 1} = \underbrace{\underbrace{X}_{N\times 3}\cdot\underbrace{\beta}_{3\times 1}}_{N\times 1} + \underbrace{\varepsilon}_{N\times 1}
\end{equation}

Recall that the solution to the least squares minimization problem
is:

\begin{equation}
\underbrace{\widehat{\beta}}_{3\times 1} = \underbrace{[\underbrace{\underbrace{X'}_{3\times N}\cdot\underbrace{X}_{N\times 3}}_{3\times 3}]^{-1}\cdot\underbrace{\underbrace{X'}_{3\times N}\cdot\underbrace{Y}_{N\times 1}}_{3\times 1}}_{3\times 1}
\end{equation}

where $\widehat{\beta}$ is our estimator of $\beta$.

In [11]:
%Estimate the betas of a linear regression 
%Here we use the '\' operator, the inverse matrix division, in
%place of the 'inv()' function

B = (X'*X)\(X'*Y)

B =

   16.3029
    0.0272
    0.7408


Now we want to generate the estimate of the $\varepsilon$, which is
$\widehat{\varepsilon}$.

Using our estimate $\widehat{\beta}$ from the above equation we can solve for $\widehat{\varepsilon}$:

\begin{equation}
\widehat{\varepsilon} = Y - X\cdot\widehat{\beta}
\end{equation}

In [8]:
%generate the estimated error
eps = Y - X*B;

To generate the goodness of fit, we first need to generate 
the following:

Predicted y, $\hat{y}$, is given by 
\begin{equation}
\hat{y} = X\cdot\widehat{\beta}
\end{equation}

The mean of our observations of y:
\begin{equation}
\bar{y} = \frac{1}{N}\sum^{N}_{i=1}y_{i}
\end{equation}

The sum of squared regressions (SSR):
\begin{equation}
SSR = \sum^{N}_{i=1}(\hat{y}_{i} - \bar{y})^{2}
\end{equation}

The sum of squared errors (SSE):
\begin{equation}
SSE = \sum^{N}_{i=1}(\widehat{\varepsilon}_{i}\cdot \widehat{\varepsilon}_{i})
\end{equation}

And the sum of squared total (SST):
\begin{equation}
SST = \sum^{N}_{i=1}(y_{i} - \bar{y})^{2}
\end{equation}

In [10]:
%generate goodness of fit:
y_bar = mean(Y)
y_hat = X*B;
SSR = sum((y_hat-y_bar).^2)
SSE = sum(eps.^2)
SST = sum((Y-y_bar).^2)

y_bar =

   20.4099


SSR =

  971.2347


SSE =

   4.2958e+03


SST =

   5.2670e+03


The formulas for the centered (c) goodness of fit:

\begin{equation}
R^{2}_{c,1} = 1 - \frac{SSE}{SST}
\end{equation}

and

\begin{equation}
R^{2}_{c,2} = \frac{SSR}{SST}
\end{equation}

These two are equivalent (when we have a constant in the model), because...

\begin{align}
SST &= SSE+SSR \Rightarrow SST-SSE=SSR \\
\\
R^{2}_{c,1} &= 1 - \frac{SSE}{SST} = \frac{SST-SSE}{SST}= \frac{SSR}{SST} = R^{2}_{c,2}
\end{align}

and the adjusted $R^{2}$ (which allows us to compare the goodness of fit between
two models estimated from the same data) is:

\begin{equation}
\overline{R}^{2} = 1-\frac{\frac{SSE}{(N-K)}}{\frac{SST}{(N-1)}}
\end{equation}

In [24]:
%Goodness of fit:
R_1 = 1-(SSE/SST)
R_2 = SSR/SST

%adjusted measure
R_bar = 1 - (SSE/(N-K))/(SST/(N-1))

R_1 =

    0.1844


R_2 =

    0.1844


R_bar =

    0.1761


The formula to calculate the estimated variance of the model is given by

\begin{equation}
\widehat{\sigma}^{2} = \frac{SSE}{(N-K)} = \frac{1}{(N-K)}\underbrace{\widehat{\varepsilon}'}_{1\times N}\cdot\underbrace{\widehat{\varepsilon}}_{N\times 1}
\end{equation}

where the last equality is the same operation as SSE above in matrix algebra.

In [14]:
%generate the estimated variance
var_hat = SSE/(N-K)

var_hat =

   21.8060


We can test that we have the desired outcome:
\begin{equation}
\text{E} (\underbrace{\underbrace{X'}_{3\times N}\cdot\underbrace{\widehat{\varepsilon}}_{N\times 1}}_{3\times 1})=\underbrace{0}_{3\times 1}
\end{equation}

In [15]:
X_eps=X'*eps %dimX: 200x3 dim_eps:200x1 X_eps=3x200 * 200x1
E=mean(X_eps)

X_eps =

   1.0e-10 *

   -0.0027
   -0.0659
   -0.1192


E =

  -6.2634e-12


To obtain the variance-covariance matrix, we need:

\begin{equation}
\underbrace{\widehat{V}}_{3\times 3} = \underbrace{\widehat{\sigma}^{2}}_{scalar}[\underbrace{\underbrace{X'}_{3\times N}\cdot\underbrace{X}_{N\times 3}}_{3\times3}]^{-1}
\end{equation}

In [17]:
%Var-Covar matrix
Var_Cov = var_hat*inv(X'*X) %dim X'X: 3x200*200*3=3x3

Var_Cov =

    0.4881   -0.0022   -0.0690
   -0.0022    0.0002   -0.0001
   -0.0690   -0.0001    0.0136


We need the above diagonals, which are the variances of each $\widehat{\beta}_{k}$.

Recall that the formula for the standard error is given by:

\begin{equation}
SE_{\widehat{\beta}_{k}} = \sqrt{\frac{\hat{\sigma}^{2}}{\underbrace{\underbrace{X_{k}'}_{1\times N}\cdot \underbrace{X_{k}}_{N\times1}}_{1\times1}}}
\end{equation}

We use the diag() function in matlab to get this directly.

In [19]:
%Standard errors of the betas
SE_B = sqrt(diag(Var_Cov))

SE_B =

    0.6986
    0.0155
    0.1167


For the t-statistic we do the following, recall that the formula under the null:
\begin{equation}
H_{O}:\quad\widehat{\beta}_{0,k}=0
\end{equation}

is the following:
\begin{equation}
t_{\widehat{\beta}_{k}} = \frac{\widehat{\beta}_{k}-\widehat{\beta}_{0,k}}{SE_{\widehat{\beta}_{k}}}
\end{equation}

In [20]:
%t-statistic for inference testing:
%in matrix algebra with MATLAB we can do this with 
%the './' operator (element-by-element division)
%Divide element 1 of B by element 1 of SE_B, and so on
t = B./SE_B

t =

   23.3356
    1.7473
    6.3468


For confidence interval, we will need to find the t-statistic of the model (different) from the above t statistic.

We will do a two sided t-interval at an $\alpha=0.05$ confidence means that wee need the $t_{\alpha=0.025}$ and $t_{\alpha=0.975}$

Recall that a confidence interval is given by:

\begin{equation}
\left[\widehat{\beta}_{k} - t_{\alpha=\frac{0.05}{2}}\cdot SE_{\widehat{\beta}_{k}}\quad,\quad\widehat{\beta}_{k} + t_{\alpha=\frac{0.05}{2}}\cdot SE_{\widehat{\beta}_{k}} \right]
\end{equation}

In [32]:
%Generate a matrix with the confidence intervals

%first, grab the two-sided t test statistic for the data
ts = tinv([0.025  0.975], N-1)

%calculate the confidence intervals (concatinate vector math): 
% Conf_Inv = [Lower Interval, Beta Estimate, Upper Interval]
Conf_Int = [B+ts(1).*SE_B, B, B+ts(2).*SE_B ]

ts =

   -1.9720    1.9720


Conf_Int =

   14.9252   16.3029   17.6806
   -0.0035    0.0272    0.0578
    0.5106    0.7408    0.9710


For the F-Statistic of global significance, we need to define the restricted model:

\begin{equation}
H_{O}:\quad \widehat{\beta}_{k=2}=\widehat{\beta}_{k=3} =0
\end{equation}

To test this we need to define the $q\times K$ matrix R that defines the restrictions:

\begin{equation}
R\cdot \widehat{\beta}=r
\end{equation}

Since our null hypothesis posits that $\widehat{\beta}_{k=2}$ and $\widehat{\beta}_{k=3}$ are equal to 0, we need to define the matrix that gives us $q=2$ rows, one for each restriction:

\begin{equation}
R=
\begin{bmatrix}
0&1&0\\
0&0&1
\end{bmatrix}
\end{equation}

and the r matrix of what we are testing (that they equal 0):

\begin{equation}
r=
\begin{bmatrix}
0\\
0
\end{bmatrix}
\end{equation}

So our linear restrictions are:

\begin{equation}
\underbrace{
\underbrace{
\begin{bmatrix}
0&1&0\\
0&0&1
\end{bmatrix}}_{2\times3}
\cdot
\underbrace{
\begin{bmatrix}
\widehat{\beta}_{k=1}\\
\widehat{\beta}_{k=2}\\
\widehat{\beta}_{k=3}
\end{bmatrix}}_{3\times1}
}_{2\times1}
=
\underbrace{
\begin{bmatrix}
0\\
0
\end{bmatrix}}_{2\times1}
\end{equation}


In [22]:
%A simultaneous (global) test of significance:
%whether B_2=B_3=0 (basically, the constant only model is the restricted model)

%Define the linear restrictions equations:
R=[0,1,0;0,0,1];
r=[0;0];
%define rest = RB - r
rest = R*B - r;
%number of restrictions: row dim of R
q=size(R,1);

Recall that the formula for the F-Statistic is given by (do the algebra to see that the dimensions work):

\begin{equation}
F = \frac{\left(R\cdot\widehat{\beta} - r\right)'\left[\widehat{\sigma}^{2}\cdot R \cdot \left(X'\cdot X \right)^{-1}\cdot R' \right]^{-1}\left(R\cdot\widehat{\beta} - r\right)}{q} \sim F\left(q,N-K\right)
\end{equation}

In [23]:
%F-statistic calculation:
F = ( rest'/(var_hat*R/(X'*X)*R')*rest )/q

F =

   22.2699


We can use the matlab 'regress()' function to obtain the same results we have done to check our math.

To do so, we place in the matrices Y and X, as shown below.

As output, we can get several arguments. We only care about the coefficients, $\widehat{\beta}$, the confidence intervals, and the statistics.

The statistics that matlab outputs are in the following ourder:
$R^{2}$, F-stastic, p-value, and model variance $\widehat{\sigma}^{2}$.

In [28]:
%Check that our math is correct:
[Beta,CONF_INT,~,~,STATS] = regress(Y,X)

B =

   16.3029
    0.0272
    0.7408


BINT =

   14.9251   17.6806
   -0.0035    0.0578
    0.5106    0.9710


STATS =

    0.1844   22.2699    0.0000   21.8060
